In [1]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=1952d4a37d019af9688ebad7d2d4e773607cab233f5de3d0ef55868ca8edbf94
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.metrics.pairwise import cosine_similarity

#nodes
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go


import warnings

warnings.filterwarnings("ignore")

# Cleaning

In [ ]:
file = pd.read_csv('MarketBasquetAnalysis/Assignment1_Data.csv',sep=';', low_memory=False)
file['Price'] = file['Price'].replace(',','.',regex=True).astype('float')
file['Date'] = pd.to_datetime(file['Date'], format="%d.%m.%Y %H:%M")
print(len(file))
file.head()


522064


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
file = file[(file['Quantity'].between(0,100))&(file['Price'].between(0.1,20))]
file.dropna(axis=0, inplace=True, subset=['Itemname', 'CustomerID'])
len(file)

382424

In [ ]:
productos = file[['Itemname']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'itemid'})
productos

,itemid,Itemname
0,0,WHITE HANGING HEART T-LIGHT HOLDER
1,1,WHITE METAL LANTERN
2,2,CREAM CUPID HEARTS COAT HANGER
3,3,KNITTED UNION FLAG HOT WATER BOTTLE
4,4,RED WOOLLY HOTTIE WHITE HEART.
...,...,...
3810,3810,"LETTER ""U"" BLING KEY RING"
3811,3811,"LETTER ""W"" BLING KEY RING"
3812,3812,"LETTER ""Z"" BLING KEY RING"
3813,3813,PINK CRYSTAL SKULL PHONE CHARM


In [ ]:
file = file.merge(productos, on='Itemname', how='left')
len(file)

382424

In [ ]:
file['CustomerID'].unique()

array([17850., 13047., 12583., ..., 13298., 14569., 12713.])

In [ ]:
file.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country,itemid
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,4


In [ ]:
last_date_prices = file.groupby(['itemid', 'Itemname'])['Date'].idxmax()
productos = file[['itemid','Itemname','Price','Date']].loc[last_date_prices]
productos

,itemid,Itemname,Price,Date
381755,0,WHITE HANGING HEART T-LIGHT HOLDER,2.95,2011-12-08 19:55:00
333538,1,WHITE METAL LANTERN,3.75,2011-11-16 10:55:00
373406,2,CREAM CUPID HEARTS COAT HANGER,4.15,2011-12-05 14:30:00
382139,3,KNITTED UNION FLAG HOT WATER BOTTLE,4.25,2011-12-09 11:26:00
381928,4,RED WOOLLY HOTTIE WHITE HEART.,4.25,2011-12-09 09:07:00
...,...,...,...,...
373865,3810,"LETTER ""U"" BLING KEY RING",0.29,2011-12-05 15:48:00
373867,3811,"LETTER ""W"" BLING KEY RING",0.29,2011-12-05 15:48:00
373869,3812,"LETTER ""Z"" BLING KEY RING",0.29,2011-12-05 15:48:00
381283,3813,PINK CRYSTAL SKULL PHONE CHARM,0.19,2011-12-08 15:31:00


# Market Basquet Analysis

In [ ]:
most_frequent = file.groupby(['itemid'])['Quantity'].count().sort_values(ascending=False).index[0:10]
most_frequent

Index([0, 518, 137, 1365, 9, 66, 2115, 280, 250, 2995], dtype='int64', name='itemid')

In [ ]:
dfprep = file.groupby(['BillNo','itemid'])['Quantity'].sum().unstack().fillna(0)
dfprep = dfprep.applymap(lambda x: True if x>0 else False)
dfprep.head()

itemid,0,1,2,3,4,5,6,7,8,9,...,3805,3806,3807,3808,3809,3810,3811,3812,3813,3814
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,True,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536369,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
frequent_items = apriori(dfprep, min_support = 0.01, use_colnames= True)

In [ ]:
rules = association_rules(frequent_items, metric="lift",  min_threshold = 1.5)
rules.sort_values('lift',ascending=False,inplace=True)
print(len(rules))
rules['antecedents'] = rules['antecedents'].astype('str').str.replace("frozenset({","").str.replace("})","")
rules['consequents'] = rules['consequents'].astype('str').str.replace("frozenset({","").str.replace("})","")
rules.head()

986


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
578,3037,3038,0.011337,0.013774,0.010203,0.900000,65.340741,0.010047,9.862261,0.995987
579,3038,3037,0.013774,0.011337,0.010203,0.740741,65.340741,0.010047,3.813416,0.998448
576,3034,3035,0.013774,0.013547,0.010316,0.748971,55.285982,0.010130,3.929640,0.995626
577,3035,3034,0.013547,0.013774,0.010316,0.761506,55.285982,0.010130,4.135229,0.995397
610,"10, 11",1124,0.013944,0.013831,0.010203,0.731707,52.904838,0.010010,3.675722,0.994972


# Cosine Similarities

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
user_interactions = np.concatenate(([1], np.zeros(len(dfprep.iloc[0,:].values)-1)))
CustomerID = 17850.0

def get_recommeded_items(CustomerID, user_interactions, n = 2):
    dfprep = data[data['CustomerID']==CustomerID].groupby(['BillNo','Itemname'])['Quantity'].sum().unstack().fillna(0)
    dfprep = dfprep.applymap(lambda x: True if x>0 else False)
    item_similarity = cosine_similarity(dfprep.T)
    item_scores = user_interactions.dot(item_similarity)
    recommended_items = np.argsort(item_scores)[::-1][:n]
    recommended_items_name = data.columns[recommended_items]
    return recommended_items_name



In [ ]:
carrito = { 12: { 'name':"botella de agua", 'qty':3 }, 13: { 'name':"botella de wiskey", 'qty':7 }}
len(carrito)

2

# Recommender Class

In [46]:
from sklearn.metrics.pairwise import cosine_similarity
class Recommender():

    def __init__(self, transaction_df: pd.DataFrame, products_df: pd.DataFrame, rules = None):
        self.transaction_df = transaction_df
        self.products_df = df_products
        if rules:
            self.rules = rules
        else:
            try:
                self.calculate_rules()
            except:
                print('DataFrame provided is not in the correct format')
                raise Exception("Couldn't initialize Recommender")

    def calculate_rules(self):
        try:
            self.most_frequent = self.transaction_df.groupby(['itemid'])['Quantity'].count().sort_values(ascending=False).index[0:10]
            # Calculate dfprep
            self.dfprep = self.transaction_df.groupby(['BillNo','itemid'])['Quantity'].sum().unstack().fillna(0)
            self.dfprep = self.dfprep.applymap(lambda x: True if x>0 else False)
            frequent_items = apriori(self.dfprep, min_support = 0.01, use_colnames= True)
            # Calculate rules
            self.rules = association_rules(frequent_items, metric="lift",  min_threshold = 1.5)
            self.rules.sort_values('lift',ascending=False,inplace=True)
            #self.rules['antecedents'] = self.rules['antecedents'].astype('str').str.replace("frozenset({","").str.replace("})","")
            #self.rules['consequents'] = self.rules['consequents'].astype('str').str.replace("frozenset({","").str.replace("})","")
        except Exception as e:
            print('Unkown Error when calculating rules')
            raise e
        return True

    def new_client_req(self,carrito):
        try:
            itemid = self.calculate_itemids(carrito)
            print(itemid)
            recommendation = self.rules[self.rules['antecedents'].isin(itemid.astype(str))]
            recommendation = recommendation.sort_values(by='lift',ascending=False).iloc[0:len(recommendation),1].values.tolist()
            recommendation = np.array([element for element in recommendation if ',' not in str(element)])
            recommendation = np.concatenate([recommendation,np.array([i for i in self.most_frequent if i not in recommendation ])])[0:10]
        except Exception as e:
            print('Unkown Error processing when pulling rules for new clients recommended items')
            raise e
        return recommendation

    def calculate_user_interactions(self, dfprep, carrito):
        tmp = pd.DataFrame(columns=dfprep.columns,index=[0])
        tmp.fillna(0,inplace=True)
        if len(carrito)==0:
            return np.zeros(len(dfprep.columns))
        else:
            for i in carrito:
                tmp.loc[0,str(i)]=1

            print(tmp.loc[0,:].values)
            return tmp.loc[0,:].values[len(carrito):]

    def calculate_itemids(self, carrito):
        tmp = []
        if len(carrito)>0:
            for i in carrito:
                tmp += [i]
        return np.array(tmp)

    def get_recommended_items(self, customer_id,carrito, n = 10):
        dfprep = self.transaction_df[self.transaction_df['CustomerID']==customer_id].groupby(['BillNo','itemid'])['Quantity'].sum().unstack().fillna(0)
        try:
            df_client = self.transaction_df[self.transaction_df['CustomerID']==customer_id].groupby(['BillNo','itemid'])['Quantity'].sum().unstack().fillna(0)
            df_client = df_client.applymap(lambda x: True if x>0 else False)
            item_similarity = cosine_similarity(df_client.T)
            user_interactions = self.calculate_user_interactions(df_client,carrito)
            print(user_interactions.shape)
            print(item_similarity)
            item_scores = user_interactions.dot(item_similarity)
            recommended_items = np.argsort(item_scores)[::-1][:n]
            recommended_items_name = dfprep.columns[recommended_items]
        except Exception as e:
            print('Unkown Error processing when getting recommended items')
            raise e
        return recommended_items_name

    def get_product(self,customer_id,carrito=[]):
        try:
            clients = self.transaction_df['CustomerID'].unique()
            if customer_id in clients:
                recomendation = self.get_recommended_items(customer_id,carrito)
            else:
                if len(carrito)==0:
                    recomendation = np.array(self.most_frequent)
                else:
                    recomendation = self.new_client_req(carrito)
            products = []
            for id in recomendation:
                products.append(self.build_product_json(int(id)))
            return products
        except  Exception as e:
            print('Unkown Error when calculating rules')
            raise e

    def build_product_json(self, product_id):
      product_info = self.products_df[ self.products_df["itemid"] == product_id].iloc[0]
      return {
          'product_id': product_info["itemid"],
          'product_name': product_info["Itemname"],
          'price': product_info["Price"],
          'image_url': product_info["urls"],
      }

In [3]:
df_file = pd.read_csv('files.csv', index_col=0)
df_products = pd.read_csv('products.csv', index_col=0)

In [ ]:
r = Recommender(transaction_df=df_file, products_df=df_products)

In [ ]:
itemid = 3037
r.new_client_req()

In [ ]:
customer_id = 17850.0
carrito = { 55: { 'name':"botella de agua",'qty':3}}
r.get_recommended_items(customer_id,carrito)


In [ ]:
r.get_product(customer_id,carrito=carrito)
# 55, 54

[3037]


array([   0.,  518.,  137., 1365.,    9.,   66., 2115.,  280.,  250.,
       2995.])

# DEMOS

In [ ]:
#  Nuevo Cliente

In [ ]:
customer_id = 123
carrito = { }
type(r.get_product(customer_id,carrito=carrito))


list

In [ ]:
#  Nuevo Cliente - comprando

In [ ]:
customer_id = 123
carrito = { 3037: { 'name':"botella de agua",'qty':3}}
type(r.get_product(customer_id,carrito=carrito))


[3037]


list

In [ ]:
#  Viejo Cliente

In [ ]:
customer_id = 17850.0
carrito = {}
type(r.get_product(customer_id,carrito=carrito))


list

In [ ]:
#  Viejo Cliente - comprando

In [ ]:
customer_id = 17850.0
carrito = { 55: { 'name':"botella de agua",'qty':3}}
r.get_product(customer_id,carrito=carrito)


get_recommended_items recom
Int64Index([55, 54, 53, 0], dtype='int64', name='itemid')


[{'product_id': 55,
  'product_name': 'English Muffin with Sausage, Egg & Cheese',
  'price': 2.95,
  'image_url': 'https://www.7-eleven.com/_next/image?url=https%3A%2F%2Fimages.contentstack.io%2Fv3%2Fassets%2Fbltbb619fd5c667ba2d%2Fblta4ed610d7f96ce55%2F6144f01cf725af79f53fc8c1%2Fenglish-muffin-sausage-egg-cheese.jpg&w=3840&q=75'},
 {'product_id': 54,
  'product_name': 'Strawberry Cup',
  'price': 2.95,
  'image_url': 'https://www.7-eleven.com/_next/image?url=https%3A%2F%2Fimages.contentstack.io%2Fv3%2Fassets%2Fbltbb619fd5c667ba2d%2Fbltb2e8258d47a6c71c%2F60ca620bae0d50495b4f6fa7%2FStrawberry_Fruit_Cup.jpg&w=3840&q=75'},
 {'product_id': 53,
  'product_name': 'CLIF Bar Chocolate Chip Energy Bar Minis',
  'price': 8.95,
  'image_url': 'https://target.scene7.com/is/image/Target/GUEST_a7a9491c-5b43-4c74-a645-c34a06f31d2c?wid=1200&hei=1200&qlt=80&fmt=webp'},
 {'product_id': 0,
  'product_name': 'American Coffee',
  'price': 2.95,
  'image_url': 'https://www.7-eleven.com/_next/image?url=https

In [ ]:
# 17850., 13047., 12583., ..., 13298., 14569., 12713.

# Connection with Anvil

In [12]:
df_file = pd.read_csv('files.csv')
df_products = pd.read_csv('products.csv')

In [47]:
import anvil.server
import json

recommender = Recommender(transaction_df=df_file, products_df=df_products)
product_q = []

anvil.server.connect("server_TOXF5MKYG6XEKT6T6NSLGJ4W-UKESBXZIBNP534UW")

In [ ]:
@anvil.server.callable
def get_recomendations(user_id, cart):
    cart = json.loads(cart)
    print('Im alive, args')
    print(user_id, cart)
    product_q = recommender.get_product(user_id, carrito=cart)
    print(product_q)

    # Send top 4 products on q
    return product_q

anvil.server.wait_forever()